In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler_feature = StandardScaler()

df = pd.read_csv("Temp2.csv", index_col=0)

df = df[1:-1]

for col in df.columns:
    if "Close" in col or "Low" in col or "High" in col:
        if "Change" not in col:
            for idx in df[col].index:
                if idx < 748:
                    df.loc[idx, f"Next_{col}"] = df.loc[idx+1, col]

df = df[1:-1]

X = df
for col in X.columns:
    if "Target" in col or "Spread" in col or "Change" in col or "Open" in col or "Date" in col or "Next" in col:
        if "Open_Interest_All" != col:
            X = X.drop([col], axis=1)

    X_scaler = scaler_feature.fit_transform(X)

y = df.Target_Open_Interest_All

LEN_INPUT = len(X.columns)
LEN_OUTPUT = 1 #len(y.columns)

X = np.array(X)
y = np.array(y)


In [ ]:
X

In [ ]:
import keras
import os
#os.environ["KERAS_BACKEND"] = "tensorflow"
# from keras_tuner_cv.outer_cv import OuterCV
# from keras_tuner_cv.inner_cv import inner_cv
# from keras_tuner_cv.utils import pd_inner_cv_get_result
# from sklearn.model_selection import KFold
from tensorflow.keras import layers
from keras_tuner import Objective
from keras import regularizers
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense
from sklearn import model_selection
from sklearn.datasets import make_regression
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler_feature = StandardScaler()
scaler_target = StandardScaler()


def reg_wrapper(type, value):
    if type == 'l2':
        return regularizers.l2(value)
    if type == 'l1':
        return regularizers.l1(value)


callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, start_from_epoch=0, restore_best_weights=True)

X_scaler = scaler_feature.fit_transform(X)
#y_scaler = scaler_target.fit_transform(y.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, test_size=0.3, shuffle=True, random_state=42)


def build_model(hp):
    #num_layers = hp.Int('num_layers', min_value=1, max_value=2, step=1)
    reg = reg_wrapper(hp.Choice('type', ['l2']), hp.Choice('reg_value', [0.00001]))
    input_layer = keras.Input(shape=(LEN_INPUT,), )
    model = keras.Sequential()
    model.add(input_layer)
    model.add(layers.Dense(units=hp.Int(f'units_1', min_value=2048, max_value=10240, step=1024), kernel_initializer='glorot_uniform', activation='relu', activity_regularizer=reg))
    # for i in range(num_layers):
    #     if i == 0:
    #         model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=10240, step=256), kernel_initializer='glorot_uniform', activation='relu', activity_regularizer=reg))
    #     if i == 1:
    #         model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=2048, step=32), kernel_initializer='glorot_uniform', activation='relu', activity_regularizer=reg))
        # if i == 2:
        #     model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=2048, step=128), kernel_initializer='glorot_uniform', activation='relu', activity_regularizer=reg))

    model.add(layers.Dense(LEN_OUTPUT, activation='linear'))

    #hp_loss = hp.Choice("loss", values=['mae', 'mse'])
    hp_learning_rate = hp.Choice('learning_rate', values=[0.00001])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='mae', metrics=['mean_absolute_error'])
    
    return model

build_model(kt.HyperParameters())


tuner = kt.GridSearch(
    hypermodel = build_model,
    objective=Objective("val_mean_absolute_error", 
                        direction="min"),
    directory="Keras_tuner_dir",
    project_name="Keras_tuner_Demo", 
    overwrite=True,
    seed=42,
    max_trials=2500,
    
)

tuner.search(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=1000000,
    callbacks=[callback],
    verbose=True,
    batch_size=len(y)
    
)

best_hps= tuner.get_best_hyperparameters(1)[0]
print(best_hps)

best_model = tuner.get_best_models(num_models=1)[0]

3697 - 9728

In [ ]:
models = tuner.get_best_models(num_models=2)  # Gets the top 2 models
best_model = models[0]
best_model.summary()

In [ ]:
tuner.results_summary()

In [ ]:
best_model.save("my_model.keras")

In [ ]:
import datetime
log_dir = "logs/fit/"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, start_from_epoch=0, restore_best_weights=True)

input_layer = keras.Input(shape=(LEN_INPUT,), )
model = keras.Sequential()
model.add(input_layer)
model.add(layers.Dense(9728, kernel_initializer='glorot_uniform', activation='relu', kernel_regularizer=keras.regularizers.L2(l2=0.00001)))
model.add(layers.Dense(LEN_OUTPUT, activation='linear'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001),
                  loss='mae', metrics=['mean_absolute_error'])

model.fit(x=X_train, 
          y=y_train, 
          epochs=100000, 
          validation_data=(X_test, y_test), 
          callbacks=[tensorboard_callback, callback],
          verbose=True)

In [ ]:
model.summary()


In [ ]:
import pandas as pd
import numpy as np
import keras
from tensorflow.keras import layers
from keras_tuner import Objective
from keras import regularizers
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense
from sklearn import model_selection
from sklearn.datasets import make_regression
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler_feature = StandardScaler()
scaler_target = StandardScaler()


callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, start_from_epoch=0, restore_best_weights=True)

df = pd.read_csv("Temp2.csv", index_col=0)
df = df.drop(["Date"], axis=1)
df = df[1:-1]
df = df.sample(frac = 1, random_state=42)
feature_col_name = []
LEN_INPUT = 0
LEN_OUTPUT = 0
for col in df.columns:
    if "Target" not in col or "Spread" in col:
        LEN_INPUT += 1
    if "Target" in col or "Spread" in col:
        feature_col_name.append(col)
        LEN_OUTPUT += 1

predicted_df = pd.DataFrame(columns=feature_col_name)



for i in range(0, len(df), 10):
    input_layer = keras.Input(shape=(LEN_INPUT,), )
    model = keras.Sequential()
    model.add(input_layer)
    model.add(layers.Dense(9728, kernel_initializer='glorot_uniform', activation='relu', kernel_regularizer=keras.regularizers.L2(l2=0.00001)))
    model.add(layers.Dense(LEN_OUTPUT, activation='linear'))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001),
                    loss='mae', metrics=['mean_absolute_error'])

    print(i, "!!!!!!!!!!!")
    test_df = df[i:i+10]
    temp_df1 = df[:i]
    temp_df2 = df[i+10:]
    frames = [temp_df1, temp_df2]
    temp_df = pd.concat(frames)

    X = temp_df
    for col in X.columns:
        if "Target" in col:
            X = X.drop([col], axis=1)

    y = temp_df
    for col in y.columns:
        if "Target" not in col:
            y = y.drop([col], axis=1)

    for col in test_df.columns:
        if "Target" in col:
            test_df = test_df.drop([col], axis=1)

    test_data = np.array(test_df)
    X = np.array(X)
    y = np.array(y)

    
    X_scaler = scaler_feature.fit_transform(X)
    test_data_scaler = scaler_feature.transform(test_data)
    y_scaler = scaler_target.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X_scaler, y_scaler, test_size=0.3, shuffle=True, random_state=42)

    model.fit(x=X_train, 
            y=y_train, 
            epochs=100000, 
            validation_data=(X_test, y_test), 
            callbacks=[callback])
    
    predicted_t = model.predict(test_data_scaler)

    predicted = pd.DataFrame(scaler_target.inverse_transform(predicted_t), columns=feature_col_name)
    predicted.index = list(test_df.index)
    frames = [predicted, predicted_df]
    predicted_df = pd.concat(frames)

In [ ]:
predicted_df

In [5]:
predicted_df.to_csv('gen_df.csv')

In [ ]:
predicted

In [6]:
predicted.index = list(test_df.index)

In [ ]:
test_df = df[i:i+10]
for col in test_df.columns:
    if "Target" not in col:
        test_df = test_df.drop([col], axis=1)
test_df

In [ ]:
test_df.index

In [ ]:
tensorboard --logdir logs/fit

In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv("Temp2.csv", index_col=0)
df = df.drop(["Date"], axis=1)
df = df[1:-1]
X = df
for col in X.columns:
    if "Target" in col or "Spread" in col or "Change" in col or "Open" in col:
        if "Open_Interest_All" != col:
            X = X.drop([col], axis=1)

y = df.Target_Open_Interest_All



In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("gen_df3.csv", index_col=0)
df = df.sort_index()
df = df[:-1]

X = df
for col in X.columns:
    if "Next" in col or ("Target" not in col and "Close" not in col and "Low" not in col and "High" not in col):
        X = X.drop([col], axis=1)

y = df.Next_Close_0



In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler_feature = StandardScaler()
#scaler_target = StandardScaler()

X_scaler = scaler_feature.fit_transform(X)
#y_scaler = scaler_target.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, test_size=0.05, shuffle=True, random_state=42)

nn = MLPRegressor(verbose=False)

hyperparameters = dict(hidden_layer_sizes=[200, 1000, 3000, 5000, 10000], solver=['adam'],
                     alpha=[0.001, 0.0001, 0.00001], tol=[0.00001],
                     learning_rate=['adaptive', 'constant'], learning_rate_init=[0.01, 0.001, 0.0001, 0.00001],
                     max_iter=[500, 1000, 3000, 5000], early_stopping=[True],
                     n_iter_no_change=[10])

grid_search = GridSearchCV(nn, hyperparameters, n_jobs=-1, cv=5, verbose=2, error_score='raise', scoring="neg_mean_absolute_error", return_train_score=True)


hyperparameters_tuning = grid_search.fit(X_train, y_train)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_

print(tuned_model.predict(X_test))

In [ ]:
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

In [ ]:
y_test

In [28]:
predicted = pd.DataFrame(tuned_model.predict(X_test))

In [ ]:
predicted

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
testScore = mean_absolute_error(tuned_model.predict(X_test), y_test)
testScore

In [ ]:
trainScore = mean_absolute_error(tuned_model.predict(X_train), y_train)
trainScore

In [ ]:
import matplotlib.pyplot as plt
res = tuned_model.predict(X_test)
plt.plot(res)
#plt.plot(trainPredictPlot)
plt.plot(y_test.values)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler_feature = StandardScaler()

df = pd.read_csv("Temp2.csv", index_col=0)
#df = df.drop(["Date"], axis=1)
df = df[1:-1]
gen_df = df.copy()
for col in gen_df.columns:
    if "Target" in col or "Spread" in col or "Change" in col or "Open" in col:
        if "Open_Interest_All" != col:
            gen_df = gen_df.drop([col], axis=1)

X = df
for col in X.columns:
    if "Target" in col or "Spread" in col or "Change" in col or "Open" in col or "Date" in col:
        if "Open_Interest_All" != col:
            X = X.drop([col], axis=1)

for col in df.columns:
    if "Target" in col and "Spread" not in col and "Change" not in col:
        for idx in range(X.index.min(), ((X.index.max()//50+1)*50), 50):
            print(col, idx)
            test_df = X[idx:idx+50]
            temp_df1 = X[:idx]
            temp_df2 = X[idx+50:]
            frames = [temp_df1, temp_df2]
            temp_df = pd.concat(frames)
            X_scaler = scaler_feature.fit_transform(temp_df)

            #test_df = df[col][idx:idx+10]
            temp_df1 = df[col][:idx]
            temp_df2 = df[col][idx+50:]
            frames = [temp_df1, temp_df2]
            y = pd.concat(frames)

            X_train, _, y_train, _ = train_test_split(X_scaler, y, test_size=0.001, shuffle=True, random_state=42)
            nn = MLPRegressor(alpha=1e-05, early_stopping=False, hidden_layer_sizes=5000, learning_rate='adaptive', learning_rate_init=0.001, max_iter=10000, n_iter_no_change=10, solver='adam', tol=1e-05, verbose=False)
            nn.fit(X_train, y_train)
            pred = nn.predict(test_df)
            for i in range(len(pred)):
                gen_df.loc[idx+i, col] = pred[i]

            gen_df.to_csv('gen_df2.csv')


In [ ]:
df

## Это запустить



In [ ]:
import numpy as np
import keras
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error


def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

# fix random seed for reproducibility
tf.random.set_seed(7)
keras.utils.set_random_seed(42)
# load the dataset
dataframe = pd.read_csv("Temp2.csv", index_col=0)

feature_col_name = []
for col in dataframe.columns:
	if "Target" not in col and "Spread" not in col and "Change" not in col and "Open" not in col and "High" not in col and "Low" not in col and "Close" not in col and "Date" not in col or col == "Open_Interest_All":
		feature_col_name.append(col)
#predicted_df = pd.DataFrame(columns=feature_col_name, index=list(dataframe.index)[99:])
predicted_df = pd.read_csv('gen_df2.csv', index_col=0)
print(feature_col_name)
for col in dataframe.columns:
	if "Target" not in col and "Spread" not in col and "Change" not in col and "Open" not in col and "High" not in col and "Low" not in col and "Close" not in col and "Date" not in col or col == "Open_Interest_All":
		
		for i in range(0, len(dataframe)-100-1):
			if str(predicted_df.loc[100+i-1][col]) != "nan":
				continue
			print(col)
			dataset = dataframe.Open_Interest_All[:i+100]
			#print(i)
			dataset = dataset.values
			dataset = dataset.astype('float32')

			# normalize the dataset
			scaler = MinMaxScaler(feature_range=(0, 1))
			dataset = scaler.fit_transform(dataset.reshape(-1, 1))
			# split into train and test sets
			train_size = int(len(dataset) - 8)
			test_size = len(dataset) - train_size
			train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
			# reshape into X=t and Y=t+1
			look_back = 7
			trainX, trainY = create_dataset(train, look_back)
			testX, testY = create_dataset(test, look_back)
			# reshape input to be [samples, time steps, features]
			#print(scaler.inverse_transform(testX))
			#print(scaler.inverse_transform([testY]))
			trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
			testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))
			# create and fit the LSTM network
			model = Sequential()
			model.add(LSTM(4, input_shape=(look_back, 1)))
			model.add(Dense(1))
			model.compile(loss='mean_absolute_error', optimizer='adam')
			model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=0)
			# make predictions
			trainPredict = model.predict(trainX)
			testPredict = model.predict(testX)
			# invert predictions
			trainPredict = scaler.inverse_transform(trainPredict)
			trainY = scaler.inverse_transform([trainY])
			testPredict = scaler.inverse_transform(testPredict)[0][0]
			#print(testPredict)
			predicted_df.loc[100+i-1][col] = testPredict
			predicted_df.to_csv('gen_df2.csv')



In [ ]:
#print(scaler.inverse_transform(testX))
print(scaler.inverse_transform([testY]))

In [ ]:
predicted_df.to_csv('gen_df2.csv')

In [ ]:
dataframe.Open_Interest_All.tail(10)

In [ ]:
predicted_df

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import keras
import keras_tuner as kt
from keras_tuner import Objective

callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, start_from_epoch=100, restore_best_weights=True)

def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

# fix random seed for reproducibility
tf.random.set_seed(7)
# load the dataset
dataframe = pd.read_csv("Temp2.csv", index_col=0)
dataset = dataframe.Target_Open_Interest_All
dataset = dataset[1:-1].values
dataset = dataset.astype('float32')

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset.reshape(-1, 1))
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1
#look_back = 3
LOOK_BACK = 3
trainX, trainY = create_dataset(train, LOOK_BACK)
testX, testY = create_dataset(test, LOOK_BACK)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))


def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Choice('units', [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]), input_shape=(LOOK_BACK, 1)))
    model.add(Dense(1))
    model.compile(loss=hp.Choice('loss', ["mae", "mse"]), metrics=['mean_absolute_error'], optimizer='adam')
    
    return model


tuner = kt.GridSearch(
    hypermodel = build_model,
    objective=Objective("val_mean_absolute_error", 
                        direction="min"),
    directory="Keras_tuner_dir",
    project_name="Keras_tuner_Demo", 
    overwrite=True,
    seed=42,
    max_trials=2500,
    
)

tuner.search(
    trainX,
    trainY,
    validation_data=(testX, testY),
    epochs=100,
    #callbacks=[callback],
    verbose=True,
)

best_hps= tuner.get_best_hyperparameters(1)[0]
print(best_hps)

best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
# LSTM for international airline passengers problem with time step regression framing
import numpy as np
import keras
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)
# fix random seed for reproducibility
tf.random.set_seed(7)
keras.utils.set_random_seed(42)
# load the dataset
dataframe = pd.read_csv("Temp2.csv", index_col=0)
dataset = dataframe.Open_Interest_All
dataset = dataset.values
dataset = dataset.astype('float32')
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset.reshape(-1, 1))
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 1), ))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = mean_absolute_error(trainY[0], trainPredict[:,0])
print('Train Score: %.2f mean_absolute_error' % (trainScore))
testScore = mean_absolute_error(testY[0], testPredict[:,0])
print('Test Score: %.2f mean_absolute_error' % (testScore))
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions

# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset)[-100:])
#plt.plot(trainPredictPlot)
plt.plot(testPredictPlot[-100:])
plt.show()
